# Solutions to Exercises

Here are some example solutions to the exercises in the tutorial.

## Sparsity
---

The first step is to simply convert both the true and noisy signals into Fourier space.

```Python
# Define the sparse representations of the signals.
x_sparse = fft(x)
y_sparse = fft(y)
```

The noise can be estimated using `sigma_mad` using the sparse representation `y_sparse`. Then multiply by a $k$ factor of 3 or 4.

```Python
# Determine the value of lambda.
lamdba_val = 3 * sigma_mad(y_sparse) 
```

Finally, the `y_sparse` should be thresholded using `hard_thresh` as we aim to preserve the amplitude of the signal coefficients. The last trick is to remember to convert back to the direct space using the inverse Fourier transform `ifft`.

```Python
# Perform thresholding.
x_rec = ifft(hard_thresh(y_sparse, lamda_val))
```

## Denoising
---

Below is an example of the type of function you could have written. The import steps to remember were:

- to estimate the noise standard deviation (*e.g.* using `noise_est`),
- to scale $\sigma$ for each of the wavelet scales using `sigma_sclaes`,
- to assign $k$ values to get weights,
- to decompose the noisy image,
- to threshold only the wavelet scales of the decomposition using `thresh`,
- to recombine the denoised decomposition, remembering to include the coarse scale.

```python
# Write your own one denoising function here
def denoise(data, n_scales=4):

    sigma_est_scales = sigma_scales(noise_est(data), n_scales)
    weights = np.array([4] + [3] * sigma_est_scales[:-1].size) * sigma_est_scales
    data_decomp = decompose(data, n_scales)
    data_thresh = np.vstack([thresh(data_decomp[:-1].T, weights).T, 
                             data_decomp[-1, None]])

    return recombine(data_thresh)
```

Then we need to remember to use the `getdata` method from Astropy to obtain the image of Saturn.

```python
# Load the image of Saturn
saturn_noisy = getdata('./data/saturn_noisy.fits')
```

Finally, we just need to use our denoising function.

```python
# Run your own denoising function on this image 
saturn_denoised = denoise(saturn_noisy)
```